In [1]:
from load_images import load_images_from_zip
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array

2023-11-19 15:13:27.419765: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-19 15:13:27.545901: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-19 15:13:27.547710: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-19 15:13:28.738432: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
dataset_path = 'images.zip'
df = load_images_from_zip(dataset_path)
df.head()

,image_path,labels
0,apple_pie/1005649.jpg,Apple pie
1,apple_pie/1011328.jpg,Apple pie
2,apple_pie/101251.jpg,Apple pie
3,apple_pie/1014775.jpg,Apple pie
4,apple_pie/1026328.jpg,Apple pie


In [3]:
label_encoder = LabelEncoder()
df['labels_encoded'] = label_encoder.fit_transform(df['labels'])
num_classes = len(label_encoder.classes_)

In [4]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [5]:
import zipfile


def custom_data_generator(dataframe, batch_size, img_size):
    datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
    while True:
        for i in range(0, len(dataframe), batch_size):
            batch_df = dataframe.iloc[i:i+batch_size]
            batch_images = []
            batch_labels = []
            for index, row in batch_df.iterrows():
                with zipfile.ZipFile('images.zip', 'r') as zip_ref:
                    with zip_ref.open(row['image_path']) as file:
                        img = Image.open(file)
                        img = img.resize(img_size)
                        img_array = img_to_array(img)
                        batch_images.append(img_array)
                        batch_labels.append(label_encoder.transform([row['labels']])[0])
            
            # Apply data augmentation and normalization
            batch_images = datagen.flow(np.array(batch_images), shuffle=False, batch_size=batch_size).next()
            
            batch_labels = tf.keras.utils.to_categorical(batch_labels, num_classes=num_classes)
            yield batch_images, batch_labels

In [6]:
img_size = (299, 299)
batch_size = 32
epochs = 10

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(299, 299, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

2023-11-19 15:13:47.409676: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-19 15:13:47.410746: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
history = model.fit(
    custom_data_generator(train_df, batch_size, img_size),
    steps_per_epoch=len(train_df) // batch_size,
    epochs=epochs,
    validation_data=custom_data_generator(test_df, batch_size, img_size),
    validation_steps=len(test_df) // batch_size
)

Epoch 1/10


KeyboardInterrupt: 

In [11]:
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
)

In [13]:
train_generator = datagen.flow_from_dataframe(
    df,
    directory=None,
    x_col='image_path',
    y_col='labels',
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    subset='training',
)

Found 0 validated image filenames belonging to 0 classes.


/home/manasi/.local/lib/python3.10/site-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 101000 invalid image filename(s) in x_col="image_path". These filename(s) will be ignored.
  warnings.warn(
